# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

### Подготовка данных

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.core.display import HTML

In [38]:
# в файле lit.txt несколько рандомных русских романов
# отсюда https://github.com/dachelnokova/russian-novels/tree/main/texts

# сам файл тут https://github.com/dachelnokova/nlp-homeworks-2023/blob/main/lit.txt
txt = open('lit.txt').read()

In [3]:
print("Длина", len(txt))

Длина 13247168


In [4]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [word for word in normalized_text if not any(char.isdigit() or char in ['—', '»'] for char in word)]
    return normalized_text

In [5]:
norm_txt = normalize(txt)

In [6]:
print("Длина корпуса в токенах -", len(norm_txt))
print("Уникальных токенов -", len(set(norm_txt)))

Длина корпуса в токенах - 2039153
Уникальных токенов - 127476


In [7]:
# посчитаем, сколько раз встречаются слова и выведем самые частотные
from collections import Counter
vocab_txt = Counter(norm_txt)
vocab_txt.most_common(10)

[('и', 95281),
 ('в', 46973),
 ('не', 45077),
 ('что', 35806),
 ('он', 32358),
 ('на', 28491),
 ('с', 24866),
 ('я', 23350),
 ('как', 18382),
 ('его', 16163)]

In [8]:
# сколько раз встречается слово
vocab_txt['россия']

50

In [9]:
# Для того, чтобы превратить абсолютные частоты в вероятности, разделим на общее число слов в каждом корпусе
probas_txt = Counter({word:c/len(norm_txt) for word, c in vocab_txt.items()})
probas_txt.most_common(10)

[('и', 0.046725772906692144),
 ('в', 0.02303554465996421),
 ('не', 0.02210574684685259),
 ('что', 0.01755925131660057),
 ('он', 0.01586835318389547),
 ('на', 0.013971977580887751),
 ('с', 0.012194278702971283),
 ('я', 0.011450832772234354),
 ('как', 0.009014527110030488),
 ('его', 0.00792633019690038)]

In [10]:
from nltk.tokenize import sent_tokenize

In [11]:
# функция для создания n-грамм
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [12]:
# добавим теги начала и конца предложений
# два тега старта, потому что вероятность первого слова
    # будет рассчитываться как вероятность 3-грамма "старт-старт-первое слово" поделить на частоту биграммы "старт-старт"
sentences = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(txt)]

In [13]:
len(sentences)

148864

In [14]:
# отложим 50 первый предложений для перплексии
sentences = sentences[50:]

In [15]:
len(sentences)

148814

In [16]:
# вероятность слова C = вероятность триграммы ABC поделить на вероятность биграммы AB
# поэтому добавляем еще счетчик триграмм

unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in sentences:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence, n=2))
    trigrams.update(ngrammer(sentence, n=3))

### Создание матрицы

In [17]:
# В каждой ячееке будет лежать вероятность получить слово C, после биграммы AB. Биграмма AB в колонке, слово C – в строке


In [18]:
from scipy.sparse import lil_matrix

# Создание матрицы для триграмм, где строки - биграммы, а колонки - униграммы
matrix_trigrams = lil_matrix((len(bigrams), len(unigrams)))

In [19]:
id2bigram = list(bigrams)
bigram2id = {bigram: i for i, bigram in enumerate(id2bigram)}
id2unigram = list(unigrams)
unigram2id = {unigram: i for i, unigram in enumerate(id2unigram)}

In [20]:
for trigram in trigrams:
    word1, word2, word3 = trigram.split()
    bigram = ' '.join([word1, word2])
    # на пересечение биграммы и слова ставим вероятность встретить слово после биграммы
    matrix_trigrams[bigram2id[bigram], unigram2id[word3]] =  (trigrams[trigram]/
                                                                      bigrams[bigram])

In [21]:
print(matrix_trigrams[:5, :5].toarray())

[[0.00000000e+00 1.97965245e-02 2.24441249e-03 1.00796968e-04
  4.20726545e-02]
 [0.00000000e+00 0.00000000e+00 3.69993211e-02 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 1.60513644e-03 0.00000000e+00 4.68699839e-01
  0.00000000e+00]
 [0.00000000e+00 2.91262136e-02 3.23624595e-03 0.00000000e+00
  6.47249191e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]]


In [22]:
print("Вероятность слова 'деле' после биграммы 'на самом'      '",  trigrams['на самом деле']/bigrams['на самом'])
print("Вероятность слова 'интересном' после биграммы 'на самом'",  trigrams['на самом дне']/bigrams['на самом'])

Вероятность слова 'деле' после биграммы 'на самом'      ' 0.1935483870967742
Вероятность слова 'интересном' после биграммы 'на самом' 0.03225806451612903


In [23]:
matrix_trigrams[bigram2id['на самом'], unigram2id["деле"]]

0.1935483870967742

In [24]:
matrix_trigrams[bigram2id['на самом'], unigram2id["дне"]]

0.03225806451612903

### Генерация текста

In [25]:
def generate_text(matrix, bigram2id, id2unigram, n=100, start = '<start> <start>', do_sample=True):
    text = []  # Создаем пустой список для хранения сгенерированного текста
    current_bigram = start  # Устанавливаем текущую биграмму, с которой начнется генерация текста
    
    for i in range(n):  # для генерации каждого из n слов
        current_bigram_idx = bigram2id[current_bigram]  # получаем индекс текущей последней биграммы в матрице
        # случайным образом выбираем следующее слово, учитывая вероятности
        chosen_word_idx = np.random.choice(matrix.shape[1], p=matrix[current_bigram_idx].toarray()[0]) 
        chosen_word = id2unigram[chosen_word_idx]  # получаем слово по выбранному индексу
        text.append(chosen_word)  # добавляем слово в текст
        
        # формируем новую биграмму для следующей итерации
        current_bigram = current_bigram.split()[1] + ' ' + chosen_word
        
        if chosen_word == '<end>':  # Если выбранное слово - '<end>', значит, предложение закончилось
            current_bigram = '<start> <start>'  # Начинаем новое предложение
        
    return ' '.join(text)  # Возвращаем сгенерированный текст как строку, объединив элементы списка через пробелы


In [26]:
print(generate_text(matrix_trigrams, bigram2id, id2unigram, n=100).replace('<end>', '\n'))

прежде всего начнут рыться виноватого искать 
 там увидим 
 и все язык ему показывал 
 генеральша помолчала 
 нет я здешняя я вся холодею когда об этом он виноват но ты будто … у него нет но зато себя осужу и сожгу тебя 
 дело шло о прежних московских временах и московских знакомых николая но мужчин не замечаете … смех у дверей 
 степан аркадьич посоветовал ехать за нею вошла девушка небольшого роста в длинном пальто и с удивлением смотрел на нее 
 райский не отставал смердяков от души желаю да это было совершенно бесполезно 
 он не находил


In [27]:
print(generate_text(matrix_trigrams, bigram2id, id2unigram, n=100).replace('<end>', '\n'))

базаров как ты всё сделаешь отвечала долли обращаясь к дуняше 
 а не вынужденного не для нас показание раненного вами старика григория васильева 
 как из-за марьи генриховны вдруг поднялась вспутанная голова доктора 
 балашев обедал в клубе желчно скучать равнодушно поспорить в холостом обществе стало для него своею непроницаемостью глаза и он дико смотрел на лица мундиры и сюртуки та же улыбка глаз как и муж перекрестился 
 глядите все 
 возьмите меня отсюда чтобы мне умереть 
 что-то черное и страшное есть в погреб полезут на чердак в темноту скверное ощущение 
 понимаете нужно знать я ничего не


In [28]:
print(generate_text(matrix_trigrams, bigram2id, id2unigram, n=100).replace('<end>', '\n'))

так так 
 это большое строение 
 я поехал к парикмахеру потом завтракать потом в гимназии он не замечая генерала заглядывал в кошелек а потому насыщай их ибо имеешь права судить comme c est le médecin intime de la divine providence сказал он с неприятным чувством чего-то недоделанного сел в тени ракиты ловили удочками рыбу 
 он не мог бы отказаться 
 о своей жене 
 этим закончилось главное христианское богослужение 
 я найду сам 
 а вот и вся мысль игры пропадает 
 в которой находилась наташа эта молитва сильно подействовала на меня взглянет ажно прожжет 
 теоретически а


In [29]:
print(generate_text(matrix_trigrams, bigram2id, id2unigram, n=100).replace('<end>', '\n'))

ведь жив он жив и к князю андрею что он все-таки шел 
 продолжал спрашивать председатель 
 левин слушал их и читает ее исподтишка двум-трем благоволящим к нему друг сердечный проговорил он не может всё уже и поворот в другую дверь 
 в приемный покой вошли доктор с фельдшером это ее пианино здесь вообще много нисколько нисколько 
 маленький ростом дохтуров сидел прямо против вейротера с старательным жестом сказала наташа 
 вскрикнула опять настасья которой разговор этот доставлял по-видимому неизъяснимое блаженство 
 нет об чем умном ну я хочу оставьте 
 монах я lise 
 спросила фенечка принимая в обе


In [30]:
print(generate_text(matrix_trigrams, bigram2id, id2unigram, n=100).replace('<end>', '\n'))

самолюбия сказал левин дрожащим голосом игнатий никифорович и встал 
 вон они лежат у галерейки ждут 
 ну так воздержись от кофе не пей 
 человек ведь это произведение искусства 
 ах это вы его фигуру 
 о том что он в первый раз пьер вполне оценил наслажденье еды когда хотелось есть 
 паром причалил к другому все более и более сомневаясь в правде и свободе для красоты слога 
 но когда катя уговаривала меня заняться школой в деревне ничего не заметил впечатления какое может быть виновен перед начальством как и мы жизни теперь не доросли до них дотронуться 



### Расчет перплексии

In [31]:
def perplexity(logp, N): # принимает на вход логарифм совместной вероятности (logp) и общее число слов (N)
    return np.exp((-1/N) * logp)


def compute_join_trigram_model(text, word_counts, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>', '<start>'] + tokens + ['<end>'], n=3): # Используем функцию ngrammer для создания триграмм
        word1, word2, word3 = ngram.split()
        bigram = ' '.join([word1, word2])
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)  # Сглаживание для недостающих триграмм
    
    return prob, len(tokens)

ps_trigram = []
for sent in sent_tokenize(txt[:50]): # берем отложенные предложения (первые 50)
    prob, N = compute_join_trigram_model(sent, unigrams, bigrams, trigrams)
    if not N:
        continue
    ps_trigram.append(perplexity(prob, N))

np.mean(ps_trigram)

6160.945819408754

In [52]:
# перплексия этой модели для двача сильно меньше, всего 28.73794075859534
# первые 50 предложений тут видимо совсем не похожи на остальной текст

In [51]:
phrase = 'Офицер,  товарищ Тушина, был убит в начале  дела,  и в  продолжение  часа  из  сорока  человек  прислуги  выбыли семнадцать, но артиллеристы все так же были веселы и оживлены'
log_prob, N = compute_join_trigram_model(phrase, unigrams, bigrams, trigrams)
perplexity(log_prob, N)

6.028182153336275

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [33]:
# сделаем класс чтобы хранить каждый из лучей
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score 

In [34]:
def generate_with_beam_search(matrix, bigram2id, id2unigram, n=100, max_beams=5, start= "<start> <start>"):
    # изначально у нас один луч с заданным началом (start по дефолту)
    start_bigram = start.split()
    initial_node = Beam(sequence=start_bigram, score=np.log1p(0))
    beams = [initial_node]
    
    for i in range(n):
        # делаем n шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            #print("Current beam sequence:", beam.sequence)
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue
             # Вынимаем последнюю биграмму из последовательности и ищем ее индекс
            last_bigram = " ".join(beam.sequence[-2:])
            last_bigram_id = bigram2id.get(last_bigram)

            # посмотрим вероятности продолжений для последней биграммы в матрице
            probas = matrix.getrow(last_bigram_id).toarray()[0]

            # возьмем топ n самых вероятных продолжений
            # top_idxs = probas.argsort()[:-(max_beams+1):-1]
            top_idxs = np.random.choice(matrix.shape[1],
                                        size=min(max_beams,probas.astype(bool).sum()),
                                        p=probas, replace=False)
                
            for top_id in top_idxs: # итерируемся по индексам 
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    break
                
                # для каждого варианта продолжения создадим новую ветку, которая включает все предыдущее и новое слово
                new_sequence = beam.sequence + [id2unigram[top_id]]
                # создаем новый скор каждого луча это произведение вероятностей (или сумма логарифмов)
                new_score = beam.score + np.log1p(probas[top_id])
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        # отсортируем лучи по скору и возьмем только топ n max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
    
    # в конце возвращаем самый вероятный луч
    best_sequence = max(beams, key=lambda x: x.score).sequence

    return ' '.join(best_sequence)

In [35]:
print(generate_with_beam_search(matrix_trigrams, bigram2id, id2unigram))

<start> <start> защитник же продолжал пользоваться всеми средствами этому нововведению но дело действительно разнеслось и достигло ушей начальства уши-то ведь у гоголя из писателей говорят эта черта совершенное отсутствие кокетства особенно удивляла и потому николушка заведя своего собственного таланта ни остановиться на чем-нибудь одном мешали плотно сложиться тоске о сыне она предложила наташе говеть и говеть не так живет не для нужд своих жить а живут войска курящие трубки раскладывающие костры на сенатской площади из сенатских стульев и варящие себе есть здесь бедный но милостыни еще до пробуждения ее являлись еще новые вопросы все пытались разрешать их писали читали говорили проекты всё


In [42]:
print(generate_with_beam_search(matrix_trigrams, bigram2id, id2unigram, start= "я вас"))

я вас батюшка пригласил теперь по-домашнему совершенно этак ни с кем кроме соседа то есть прекращения отношений с благовоспитанными людьми своего привычного круга как будто отыскивая свой чепец через мельницу но другие поднятые воротники будут вас бить холодом пока вы живы продолжал он похаживая по комнате ходит да красные пятна зарделись на щеках смягчая смуглый цвет лица красиво подстриженные с сильной проседью волосы на висках и около губ ее улыбка обращенная к нему навстречу он заключил тоже что деньгу нажить любит наживает на злые проценты дает пройдоха шельма без жалости без содрогания сердца с энтузиазмом проговорил коля задрожавшим голоском побледнев и чувствуя


In [53]:
# кажется, beam search работает получше, генерирует более связные предложения